Finetune a Pretrained Hugginface model

In [1]:
#Import Libraries
import os

#Data manipulation
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split

#finetuning
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer

#Evaluation
from datasets import load_metric



Activate weight and Biases

In [ ]:
os.environ["WANDB_DISABLED"] = "false"

Load The Datasets

In [4]:
#load data
train_df=pd.read_csv('dataset\Train.csv')

#check first 5

train_df.head()


,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


AS Noticed in EDA we will drop the rows with missing values

In [9]:
#Drop null rows
train_df.dropna(axis=0,inplace=True)

#check for the presence of null values again
train_df.isna().sum()


tweet_id     0
safe_text    0
label        0
agreement    0
dtype: int64

Split the Train the dataset into training(for learning ) and evaluation(For computing metrics to capture overfitting) sets

In [10]:
train , eval = train_test_split(train_df,train_size=0.8,stratify=train_df['label'])

check train data

In [11]:
train.head()

,tweet_id,safe_text,label,agreement
8795,O1J5PFAR,Illinois Measles Count Climbs To 13 Cases: Two...,0.0,1.000000
3949,35FNMDZ2,"Measles cases in US in 2014 - 644. In 2015, 10...",-1.0,0.666667
9317,QOJ7IG8W,Important Immunizations to Consider for a Heal...,0.0,0.666667
8644,BD0AHE3X,"That's a matter of opinion, honestly! There's ...",1.0,1.000000
7315,8N23WC39,"""<> from a people nearly wiped out by disease,...",1.0,0.666667


Check eval

In [12]:
eval.head()

,tweet_id,safe_text,label,agreement
5941,7K5UFJWU,#DYK 1 child dies every 20 sec from a vaccine-...,1.0,0.666667
5362,780OYUTW,"<> Welcome to the club. It's a happy, measles...",0.0,0.666667
7327,40ZKVYM1,measles outbreak it aint no joke,1.0,0.666667
9771,8RFTGSGJ,You Won’t Believe What Obama is Forcing Public...,-1.0,0.333333
4650,3HT6ALUI,ALERT -- Health Officials Warn Of Seattle Airp...,0.0,1.000000


Check the shapes of both dataset

In [13]:
print(f'Shape of training set: {train.shape}\nShape of evaluation set: {eval.shape}')

Shape of training set: (7999, 4)
Shape of evaluation set: (2000, 4)


Save the splitted data to be loaded as huggingface dataset

In [14]:
train.to_csv('dataset\Train_subset.csv',index=False)
train.to_csv('dataset\eval_subset.csv',index=False)

In [ ]:
datasets=load_dataset('csv',data_files={'train':'dataset\Train_subset.csv','eval':'dataset\eval_subset.csv'})

Create a function to tokenize the data
create a function to transform the labels or call lable encoder


In [18]:
#Create a tokenizer instance
tokenizer=AutoTokenizer.from_pretrained('bert-base-cased')

In [19]:
#function to tokenize text 
def tokenize_data(data):
    return tokenizer(data['safe_text'],padding='max_length')


#function to transform labels
def transfom_label(data):

    #extract label
    label=data['label']
    num=0
    #create contitions
    if label==-1: #Negative
        num=0
    elif label==0: #Neutral
        num=1
    else:
        num=2      #Positive
    return {'labels':num}



Use .map to apply function to dataset, .map speeds up data processing

In [20]:
#The following columns will be removed after tokenization 
remove_columns = ['tweet_id', 'label', 'safe_text', 'agreement']

#Tokenize the text data
datasets=datasets.map(tokenize_data,batched=True)

#transform the labels
datasets=datasets.map(transfom_label,remove_columns=remove_columns)

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

# Training a Bert model


Extract the train and eval datasets from datasets

In [21]:
#extract train datasets
train_dataset=datasets['eval'].shuffle(seed=0)
#extract eval datasets
eval_dataset=datasets['eval'].shuffle(seed=0)

Loading cached shuffled indices for dataset at C:\Users\sylvester\.cache\huggingface\datasets\csv\default-620129b237d1ce84\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-46cb31b8f1381c6e.arrow


In [23]:
#set the training arguments
trainargs=TrainingArguments('training\.test',num_train_epochs=1000,evaluation_strategy="epoch",save_strategy='epoch',load_best_model_at_end=3)

#Create an instance of the bert model
model=AutoModelForSequenceClassification.from_pretrained('bert-base-cased',num_labels=3)

#create a trainer instance
trainer=Trainer(model=model,train_dataset=train_dataset,eval_dataset=eval_dataset,tokenizer=tokenizer)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`